# Делаем музыку

In [1]:
from pathlib import Path

# Путь к проекту
SOURCE_PATH = Path().absolute()
# Отладочные сообщения: True - печатать сообщения; False - не печатать
DEBUG = False
# терминатор - это сочетание 0xE и 0xF для графа де Брюйна с кол-вом брейков (концов записи аккорд) = L-1 (символ 0xE - старт записи вершины; символ 0xF = break).
TERMINATOR = 0

## 1 Генерируем графы

### 1.1 Пути

In [2]:
# Путь к директории с исходными миди
SOURCE_MIDI_FOLDER = SOURCE_PATH / "data" / "midi_sources"

# Путь к директории с результатами: графами де Брюйна
PCL_RESULT_FOLDER = SOURCE_PATH / "data" / "midi_results"

# Словарь вершин графа в формате: индекс вершины, полученный как crc32 аккордов | код вершины
VERTEX_DICTIONARY_PATH = PCL_RESULT_FOLDER / "dictionary.pcl"

### 1.2 Настройка

In [3]:
from chord_processing import ChordProcessor
from midi_processing import MidiProcessor

# Настройка графа Де Брюйна (параметр L - количество аккордов для кода вершины)
L = 3

chord_processor = ChordProcessor(L, TERMINATOR, debug=DEBUG)
midi_processor = MidiProcessor(chord_processor, VERTEX_DICTIONARY_PATH)

### 1.3 Выбираем midi-файлы и обрабатываем

In [ ]:
from midi_processing import get_filtered_files

# Файлы для обработки
midi_files: list[Path] = get_filtered_files(SOURCE_MIDI_FOLDER, '*.mid')
print(f"Файлы для обработки: {midi_files}")

# Обрабатываем файлы
midi_processor.process(midi_files, PCL_RESULT_FOLDER)

## 2 Генерируем музыку

### 2.1 Чтение графа

In [4]:
# Путь к хранилищу графов
# GRAPH_SOURCES = Path("/data/hackathon2023/pcl")
# PCL_PATH = GRAPH_SOURCES / "PianoChords_dst_l5_concatenated" #168K цепочек аккордов
# PCL_PATH = GRAPH_SOURCES / "WorldMusic_dst_l5_concatenated" #116K произведений
PCL_PATH = PCL_RESULT_FOLDER  # Path("/data/iu_home/iu6042/lab6/data/midi_results")
assert PCL_PATH.exists()

In [5]:
from music_generation import combine_pickle_files, get_files_with_params

L = 3
TONALITY = "C_major"  # TODO: tonality select ?

# Объединим графы деБрюйна
files = get_files_with_params(PCL_PATH, TONALITY, L)
df = combine_pickle_files(files)

print(f"Количество ребер в графе ДеБрюйна: {len(df)}")

Количество ребер в графе ДеБрюйна: 2693


### 2.2 Использем GPC 

In [6]:
from music_generation import GPCWrapper

SW_KERNEL_PATH = SOURCE_PATH / "lab7" / "sw-kernel" / "sw_kernel.rawbinary"
HANDLERS_PATH = SOURCE_PATH / "lab7" / "include" / "gpc_handlers.h"

generator = GPCWrapper(SW_KERNEL_PATH, HANDLERS_PATH, TERMINATOR)

In [7]:
# Максимальное количество голосов для стилистической обработки
MAX_VOICE_COUNT = 128

# Количество пройденных вершин графа ДеБрюйна (длинна произведения)
CHORD_COUNT = 2000


edges_count, origin_mid, mono_mid, voice_count = generator.run(df, CHORD_COUNT, MAX_VOICE_COUNT)

# Выводим информацию о среднем количестве ребер
randomnicity = f"{edges_count / CHORD_COUNT:.1f}"
print("Обход графа ДеБрюйна завершен, среднее количество ребер: " + randomnicity)

/dev/gpc2
{'insert_edges': 16, 'get_vertex_data': 17, 'get_first_vertex': 18, 'get_next_vertex': 19, 'get_random_vertices': 20}
Обход графа ДеБрюйна завершен, среднее количество ребер: 2.2


### 2.3 Настройки сохранения

In [8]:
from os import system as execute_command
from datetime import datetime

# Путь к директории с результирующим миди
RESULT_PATH = SOURCE_PATH / "results"

# Имя результирующего файла
date_time = datetime.now().strftime("%d.%m.%Y-%H:%M:%S")

# Создадим директорию для разделения результата на моноголосные партии
result_path = RESULT_PATH / f"random_L{L}_{date_time}_{TONALITY}_r{randomnicity}"
execute_command(f"mkdir -p {result_path}")

filename = "result"

### 2.4 Стилизация

In [9]:
# Стилизатор  # Бакалаврский проект Петра Шумнова & ИУ7 (All rights reserved) - https://github.com/MrShumnov/music-style-performer
# Путь к стилизатору
STYLE_PERFORMER = Path("/data/hackathon2023/music-style-performer")
# Путь к образцу стиля исполнения
STYLES_PATH = STYLE_PERFORMER / "styles"

MIDI_STYLE = STYLES_PATH / "debussy_prelude.mid"  # файл со стилем, переносимым на произведение/ Дебюсси - импрессионист, близко к джазовой импровизации
# MIDI_STYLE = STYLES_PATH / "scrjabin.mid"       # файл со стилем, переносимым на произведение/ Классика, романтизм, эмоциональный окрас
# MIDI_STYLE = STYLES_PATH / "rachmaninoff.mid"   # файл со стилем, переносимым на произведение/ Рахманинов - русская классика, широко и эмоционально


In [10]:
from music_generation import PerformerWrapper

merged_mid_styled = PerformerWrapper(STYLE_PERFORMER, MIDI_STYLE).stylize(mono_mid, voice_count)
merged_mid_styled.save(result_path / f"{filename}.mid")

Стилизация голоса 1 из 5
Start training
Step: 500 | total: 5.268390655517578, style: 0.14743469655513763, quality: 0.8453498482704163
Step: 1000 | total: 4.818389415740967, style: 0.13289977610111237, quality: 0.831396222114563
Step: 1500 | total: 4.51201868057251, style: 0.12100035697221756, quality: 0.8820081949234009
Step: 2000 | total: 4.2816572189331055, style: 0.11109382659196854, quality: 0.9488427639007568
Step: 2500 | total: 4.103634834289551, style: 0.10285980254411697, quality: 1.0178407430648804
Step: 3000 | total: 3.97841477394104, style: 0.09653086215257645, quality: 1.082489013671875
Step: 3500 | total: 3.8998708724975586, style: 0.09227757155895233, quality: 1.1315436363220215
Step: 4000 | total: 3.848142385482788, style: 0.08941449970006943, quality: 1.1657073497772217
Step: 4500 | total: 3.8181447982788086, style: 0.08775412291288376, quality: 1.1855212450027466
Step: 5000 | total: 3.8005216121673584, style: 0.08666253089904785, quality: 1.2006456851959229
Step: 5500 

NameError: name 'debug' is not defined

### 2.5 MP3

In [ ]:
from music_generation import MidiToMp3Converter

# Настройка темпа при генерации mp3 (выполняется в timidity)
TEMPO = 40

# Настройка длительности mp3 (максимальное время звучания в секундах). Оставшаяся часть midi в аудио не входит
MAX_DURATION = 240

midi_to_mp3 = MidiToMp3Converter(SOURCE_PATH / "midi2mp3.sh", TEMPO, MAX_DURATION)

In [ ]:
# Синтезировать звук фортепианного исполнения через Timidity
midi_to_mp3.convert("timidity_piano.cfg", result_path / f"{filename}.mid", result_path / f"{filename}_pianoforte.mp3")

In [ ]:
import numpy as np
from music_generation import apply_instruments_table

instruments_table = np.matrix([  # TODO: убрать использование numpy
    # "Название инструмента", "Номер канала", "Нижняя граница", "Верхняя граница"
    ['Guitar', 1, 40, 60],
    ['Ahh', 2, 56, 84],
    ['Violins', 3, 72, 120]
])

In [ ]:
# Синтезировать звук через Timidity
apply_instruments_table(instruments_table, merged_mid_styled).save(result_path / f"{filename}_styled.mid")
midi_to_mp3.convert("timidity.cfg", result_path / f"{filename}_styled.mid", result_path / f"{filename}_styled.mp3")

In [ ]:
# Для сравнения, синтезировать звук через Timidity для origin
apply_instruments_table(instruments_table, origin_mid).save(result_path / f"{filename}_origin.mid")
midi_to_mp3.convert("timidity.cfg", result_path / f"{filename}_origin.mid", result_path / f"{filename}_origin.mp3")